<a href="https://colab.research.google.com/github/pgordin/OptDisc2024/blob/main/Grafy_zad2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from random import random, seed

# Funkcje grafowe

In [ ]:
def print_matrix(vertices, matrix):
    '''
    wypisuje na ekranie graf podany jako macierz sąsiedztwa
    '''
    n = len(matrix)
    if (vertices is not None) and (len(vertices) == n):
        vv = vertices
    else:
        vv = range(1, n+1)
    for i in range(n):
        print(vv[i], ":", end = "")
        for j in range(n):
            if matrix[i, j]:
                print(" ", vv[j], end="")
        print("")

In [ ]:
def print_graph(graph):
    '''
    wypisuje na ekranie graf podany jako słownik w pythonie
    '''
    for v in graph:
        print(v, ":", end = "")
        for u in graph[v]:
            print("", u, end = "")
        print("")

Tworzenie i modyfikacja grafów

In [ ]:
def add_vertex(graph, vertex):
    '''
    Nowy wierzchołek do istniejącego grafu
    '''
    if vertex not in graph:
        graph[vertex] = []

def add_arc(graph, arc):
    '''
    Dodaje nowy łuk (podany jako para wierzchołków) do istniejącego grafu
    Rozważamy grafy proste, skierowane
    '''
    u, v = arc
    add_vertex(graph, u)
    add_vertex(graph, v)
    if v not in graph[u]:
        graph[u].append(v)

def add_edge(graph, edge):
    '''
    Dodaje nową krawędź (podaną jako parę wierzchołków) do istniejącego grafu
    traktując graf nieskierowany jako prosty graf skierowany, symetryczny i bez pętli.
    '''
    u, v = edge
    add_vertex(graph, u)
    add_vertex(graph, v)
    if u == v:
        raise ValueError("Pętla!")
    if v not in graph[u]:
        graph[u].append(v)
    if u not in graph[v]:
        graph[v].append(u)


Losowe grafy

In [ ]:
def random_graph(n, p):
    '''
    losowanie grafu o n wierzchołkach z p-ństwem połączenia p
    '''
    random_graph = {}
    for i in range(1, n+1):
        add_vertex(random_graph, i)
        for j in range(1, i):
            if random() < p:
                add_edge(random_graph, [i, j])
    return random_graph

Konwersja między macierzą sąsiedztwa, a słownikiem

In [ ]:
def matrix_to_dict(vertices, matrix):
    '''
    konwersja między postacią macierzową a słownikową
    '''
    n = len(vertices)
    graph = {}
    for vertex in vertices:
        graph[vertex] = []
    for i in range(n):
        for j in range(n):
            if matrix[i, j] == 1:
                graph[list(graph.keys())[i]].append(list(graph.keys())[j])
    return graph

In [ ]:
def dict_to_matrix(graph):
    '''
    konwersja między postacią słownikową a macierzową
    '''
    n = len(graph)
    matrix = np.zeros((n,n))
    vertices = list(graph.keys())
    for key in graph:
        for arc in graph[key]:
            i = list(graph.keys()).index(key)
            j = list(graph.keys()).index(arc)
            matrix[i, j] = 1
    return (vertices, matrix)

Tworzenie cyklu o n wierzchołkach

In [ ]:
def cycle(n):
    '''
    tworzenie cyklu o n wierzchołkach
    '''
    cycle = {}
    for i in range(1, n+1):
        add_vertex(cycle, i)
        cycle[i].append(i%n + 1)
    return cycle

# Wczytywanie grafów z plików

In [ ]:
def graph_from_edges(filename, directed = 0):
    '''
    wczytuje graf z pliku tekstowego (pełna ścieżka dostępu), który w każdej linii zawiera opis jednej krawędzi (pary słów),
    ewentualnie jednego wierzchołka (pojedyncze słowo). Jako wynik zwraca graf w formie listy sąsiedztwa
    '''
    graph = {}
    file = open(filename, "r")             #otwieranie pliku do odczytu
    for line in file:                      #dla każdej linii w pliku
        words = line.strip().split()       #rozbijam linię na słowa
        if len(words) == 1:                #jedno słowo = wierzchołek
            add_vertex(graph, words[0])
        elif len(words) == 2:              #więcej słów - używamy dwóch pierwszych
            if directed:
                add_arc(graph, (words[0],words[1]))
            else:
                add_edge(graph, (words[0],words[1]))
    file.close()
    return graph

def graph_to_edges(graph, filename):
    """
    Wczytuje graf z pliku zawierającego opis jednej krawędzi (pary słów), ewentualnie jednego wierzchołka
    """
    file = open(filename, "w")

    vertexes = list(graph.keys())
    print(vertexes)
    for v in graph:
        if len(graph[v]) != 0:
            for u in graph[v]:
                desc = f"{v} {u}\n"
                file.write(desc)
                if u in vertexes:
                    vertexes.remove(u)
        elif v in vertexes:
            desc = f"{v}\n"
            file.write(desc)

    file.close()

def graph_from_neighbourlist(filename):
    """
    Wczytuje graf z pliku tekstowego zawierającego listę sąsiedztwa
    """
    graph = {}
    file = open(filename, "r")
    for line in file:
        words = line.strip().split(":")
        vertex = words[0].strip()
        neighbours = words[1].strip().split()
        add_vertex(graph, vertex)
        if len(neighbours) > 0:
            for v in neighbours:
                add_arc(graph, (vertex,v))
    file.close()
    return graph

def graph_to_neighbourlist(graph, filename):
    '''
    zapisuje graf do pliku tekstowego (pełna ścieżka dostępu) jako listę sąsiedztwa
    '''
    file = open(filename, 'w')   #otwarcie pliku do zapisu
    for v in graph:
        neigh_list = f"{v}:"
        for u in graph[v]:
            neigh_list = neigh_list + f" {u}"  #u na koniec listy sąsiedztwa
        neigh_list = neigh_list + "\n"  #koniec wiersza
        file.write(neigh_list)
    file.close()



# Przykłady wykorzystania

In [ ]:
vertices = ["a", "b", "c", "d", "e", "i"]
matrix = np.array([[0,1,1,0,0,0],[1,0,1,0,0,0],[1,1,0,0,1,0],[0,0,1,0,1,1],[0,0,1,1,0,1],[1,0,1,0,1,0]])

In [ ]:
print(vertices)
print(matrix)
print("------------------------------------------------------")
print_matrix(vertices, matrix)
print("------------------------------------------------------")
print_matrix(None, matrix)

['a', 'b', 'c', 'd', 'e', 'i']
[[0 1 1 0 0 0]
 [1 0 1 0 0 0]
 [1 1 0 0 1 0]
 [0 0 1 0 1 1]
 [0 0 1 1 0 1]
 [1 0 1 0 1 0]]
------------------------------------------------------
a :  b  c
b :  a  c
c :  a  b  e
d :  c  e  i
e :  c  d  i
i :  a  c  e
------------------------------------------------------
1 :  2  3
2 :  1  3
3 :  1  2  5
4 :  3  5  6
5 :  3  4  6
6 :  1  3  5


Druga reprezentacja

In [ ]:
graph = {
    'a' :  ['b',  'c'],
    'b' :  ['a',  'c'],
    'c' :  ['a',  'b',  'e'],
    'd' :  ['c',  'e',  'i'],
    'e' :  ['c',  'd',  'i'],
    'i' :  ['a',  'c',  'e'],
    'g' :  []
}
print(graph)

{'a': ['b', 'c'], 'b': ['a', 'c'], 'c': ['a', 'b', 'e'], 'd': ['c', 'e', 'i'], 'e': ['c', 'd', 'i'], 'i': ['a', 'c', 'e'], 'g': []}


In [ ]:
print_graph(graph)

a : b c
b : a c
c : a b e
d : c e i
e : c d i
i : a c e
g :


In [ ]:
graph_to_neighbourlist(graph, "graf_1.txt")

In [ ]:
print_graph(graph_from_neighbourlist("graf_1.txt"))

a : b c
b : a c
c : a b e
e : c d i
d : c e i
i : a c e
g :


In [ ]:
%%writefile lista.txt
A B
B C
B D
D C
E
F

Writing lista.txt


In [ ]:
graph_1 = graph_from_edges("lista.txt", directed=1)

In [ ]:
graph_1

{'A': ['B'], 'B': ['C', 'D'], 'C': [], 'D': ['C'], 'E': [], 'F': []}

In [ ]:
graph_to_edges(graph_1, "lista2.txt")

['A', 'B', 'C', 'D', 'E', 'F']
